In [6]:
import numpy as np
import os 
import pandas as pd
#from pandas_profiling import ProfileReport
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn import ensemble, tree, linear_model
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from scipy.stats import pearsonr
import pyodbc
import seaborn as sns

import re
from scipy import stats
from scipy.stats import chisquare
from scipy.stats import chi2_contingency
from scipy.stats import f_oneway
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import ks_2samp
from scipy.stats import norm
#import missingno as msno
import warnings
import datetime
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
import nltk
#nltk.download('popular')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eitanb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\eitanb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
path = r"C:\Users\eitanb\Documents\GitHub\ML_Football_DS18\DATA"
path_result = r"C:\Users\eitanb\Documents\GitHub\ML_Football_DS18\result"

In [8]:
df=pd.read_csv(path_result+"/"+"after_missing_value_KNN.csv", index_col=0)

In [9]:
# Convert date column to datetime format
df["date"] = pd.to_datetime(df["date"], errors='coerce')

# Sort by date to ensure correct rolling calculations
df = df.sort_values("date")

# Create a helper function to get rolling features for each team
def add_rolling_features(df, team_col, prefix, columns, window=5):
    """
    Adds rolling average features for specified columns, grouped by team.

    Args:
    - df: original dataframe
    - team_col: column containing team ID (home or away)
    - prefix: prefix for new columns (e.g. 'home' or 'away')
    - columns: list of columns to compute rolling means on
    - window: number of past matches to average
    """
    for col in columns:
        roll_name = f"{prefix}_{col}_rolling{window}"
        df[roll_name] = (
            df
            .groupby(team_col)[col]
            .shift(1)  # only previous games
            .rolling(window=window, min_periods=1)
            .mean()
            .reset_index(level=0, drop=True)
        )
    return df

# Define features to compute rolling averages for
rolling_features = ["xGoals_h", "shots_h", "shotsOnTarget_h", "xGoals_a", "shots_a", "shotsOnTarget_a"]

# Apply rolling average for both home and away teams
df = add_rolling_features(df, "homeTeamID", "home", rolling_features, window=5)
df = add_rolling_features(df, "awayTeamID", "away", rolling_features, window=5)

# Encode result_h into numeric for win rate computation
df["home_win"] = df["result_h"].map({"W": 1, "D": 0, "L": 0})

# Compute home win rate (last 5 home games for each team)
df["home_win_rate_5"] = (
    df
    .groupby("homeTeamID")["home_win"]
    .shift(1)
    .rolling(window=5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# For away team win rate, invert result_h (away win is home loss)
df["away_win"] = df["result_h"].map({"W": 0, "D": 0, "L": 1})

# Compute away win rate (last 5 away games for each away team)
df["away_win_rate_5"] = (
    df
    .groupby("awayTeamID")["away_win"]
    .shift(1)
    .rolling(window=5, min_periods=1)
    .mean()
    .reset_index(level=0, drop=True)
)

# Show the new columns added
columns_added = [col for col in df.columns if "rolling" in col or "win_rate" in col]
df[["gameID", "homeTeamID", "awayTeamID", "date"] + columns_added].head(10)


,gameID,homeTeamID,awayTeamID,date,home_xGoals_h_rolling5,home_shots_h_rolling5,home_shotsOnTarget_h_rolling5,home_xGoals_a_rolling5,home_shots_a_rolling5,home_shotsOnTarget_a_rolling5,away_xGoals_h_rolling5,away_shots_h_rolling5,away_shotsOnTarget_h_rolling5,away_xGoals_a_rolling5,away_shots_a_rolling5,away_shotsOnTarget_a_rolling5,home_win_rate_5,away_win_rate_5
5437,6185.0,177.0,161.0,2014-08-08 19:30:00,0.612752,15.0,6.2,0.592654,8.0,2.4,1.177608,11.0,5.2,0.898379,13.4,4.6,0.4,0.4
5443,6191.0,168.0,210.0,2014-08-09 20:00:00,0.637960,12.4,4.4,0.492725,14.4,4.8,0.663808,17.6,6.4,0.519442,6.4,2.4,0.4,0.2
5444,6192.0,170.0,174.0,2014-08-09 20:00:00,1.068028,12.8,4.6,0.518463,12.2,4.4,0.871684,14.6,4.8,0.663495,7.0,3.0,0.4,0.2
5442,6190.0,166.0,176.0,2014-08-09 20:00:00,1.150806,12.8,4.0,0.374059,11.2,4.2,0.512846,18.8,7.8,0.382525,8.4,2.8,0.4,0.0
5440,6188.0,169.0,175.0,2014-08-09 20:00:00,0.941866,16.4,6.2,0.455852,9.2,2.8,0.790904,16.8,8.0,0.492996,12.0,3.2,0.6,0.2
5439,6187.0,209.0,165.0,2014-08-09 20:00:00,0.790904,18.6,7.6,0.435749,9.4,2.8,0.451528,15.6,7.4,0.507669,12.2,3.0,0.6,0.2
5438,6186.0,162.0,164.0,2014-08-09 20:00:00,0.952128,16.2,6.8,0.577981,7.8,2.6,0.899550,13.2,6.0,0.898379,13.2,4.0,0.4,0.2
5441,6189.0,160.0,180.0,2014-08-09 20:00:00,0.941866,16.0,5.8,0.212896,10.6,3.6,0.790904,16.8,7.2,0.389863,8.8,2.8,0.6,0.2
5445,6193.0,178.0,163.0,2014-08-10 16:00:00,1.107160,13.0,5.0,0.381547,15.6,5.8,0.788906,15.4,5.0,0.633838,7.2,3.2,0.4,0.2
5446,6194.0,171.0,179.0,2014-08-10 20:00:00,1.036978,14.0,6.0,0.794271,14.6,5.4,0.788906,18.2,5.8,0.680484,7.8,3.2,0.4,0.2


In [11]:
df_feature_EN=df.merge(df[["gameID", "homeTeamID", "awayTeamID", "date"] + columns_added],how='left', on=["gameID", "homeTeamID", "awayTeamID", "date"])

In [14]:
df_feature_EN.columns

Index(['gameID', 'leagueID', 'season', 'date', 'homeTeamID', 'awayTeamID',
       'homeGoals', 'awayGoals', 'homeGoalsHalfTime', 'awayGoalsHalfTime',
       'xGoals_h', 'shots_h', 'shotsOnTarget_h', 'deep_h', 'ppda_h', 'fouls_h',
       'corners_h', 'yellowCards_h', 'redCards_h', 'teamgoals_h', 'ownGoals_h',
       'total_assists_h', 'total_xAssists_h', 'total_key_passes_h',
       'total_xGoalsChain_h', 'total_xGoalsBuildup_h', 'total_blocked_shots_h',
       'total_saved_shots_h', 'xGoals_a', 'shots_a', 'shotsOnTarget_a',
       'deep_a', 'ppda_a', 'fouls_a', 'corners_a', 'yellowCards_a',
       'redCards_a', 'teamgoals_a', 'ownGoals_a', 'total_assists_a',
       'total_xAssists_a', 'total_key_passes_a', 'total_xGoalsChain_a',
       'total_xGoalsBuildup_a', 'total_blocked_shots_a', 'total_saved_shots_a',
       'result_h', 'game_year', 'game_month', 'game_day', 'homeGoals_cat',
       'awayGoals_cat', 'homeGoalsHT_cat', 'awayGoalsHT_cat', 'redCards_h_cat',
       'redCards_a_cat', '

In [10]:
#checking values counts against all the numeric data
for col in df.columns:
    print(f"\nColumn: {col}")
    print(df[col].value_counts(dropna=False))


Column: gameID
gameID
6185.0     1
10878.0    1
10185.0    1
10869.0    1
10863.0    1
          ..
2790.0     1
2758.0     1
2791.0     1
2759.0     1
16135.0    1
Name: count, Length: 12680, dtype: int64

Column: leagueID
leagueID
1.0    2660
4.0    2660
2.0    2660
5.0    2558
3.0    2142
Name: count, dtype: int64

Column: season
season
2014.0    1826
2015.0    1826
2017.0    1826
2018.0    1826
2020.0    1826
2016.0    1825
2019.0    1725
Name: count, dtype: int64

Column: date
date
2019-05-12 14:00:00    13
2016-04-20 22:45:00    12
2016-05-08 19:00:00    12
2021-05-23 19:00:00    11
2017-05-20 20:00:00    11
                       ..
2015-11-29 21:00:00     1
2015-11-29 21:15:00     1
2015-11-29 23:30:00     1
2018-04-18 19:30:00     1
2014-08-08 19:30:00     1
Name: count, Length: 6834, dtype: int64

Column: homeTeamID
homeTeamID
74.0     133
96.0     133
78.0     133
101.0    133
106.0    133
        ... 
173.0     19
208.0     19
260.0     19
234.0     17
262.0     17
Name: c